# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>


## 🗒️ This notebook is divided into the following sections:

1. Load batch data.
2. Predict using model from Model Registry.

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import joblib
import inspect 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

%config InlineBackend.figure_format='retina'
%matplotlib inline

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style="color:#ff5f27;"> ⚙️ Feature View Retrieval</span>


In [ ]:
# Retrieve the 'electricity_feature_view' feature view
feature_view = fs.get_feature_view(
    name='electricity_feature_view',
    version=1,
)

## <span style="color:#ff5f27;">🗄 Model Registry</span>


In [ ]:
# Retrieve the model registry
mr = project.get_model_registry()

## <span style="color:#ff5f27;"> 📮 Retrieving model from Model Registry </span>

In [ ]:
# Retrieving the TensorFlow model from the Model Registry
model = mr.get_model(
    name="electricity_price_prediction_model", 
    version=1,
)

# Downloading the model to a local directory
model_dir = model.download()

# Loading the saved TensorFlow model
loaded_model = tf.saved_model.load(model_dir)

# Accessing the serving function of the loaded model
serving_function = loaded_model.signatures["serving_default"]

## <span style="color:#ff5f27;"> ✨ Load Batch Data</span>

In [ ]:
import datetime

# Calculating the start date as 5 days ago from the current date
start_date = datetime.datetime.now() - datetime.timedelta(days=5)

# Converting the start date to a timestamp in milliseconds
start_time = int(start_date.timestamp()) * 1000

In [ ]:
# Initializing batch scoring
feature_view.init_batch_scoring(1)

# Retrieving batch data from the feature view starting from the specified start time
df = feature_view.get_batch_data(
    start_time=start_time,
)

In [ ]:
# Sorting the DataFrame based on the 'timestamp' column
df.sort_values(["timestamp"], inplace=True)

# Dropping the 'day' and 'timestamp' columns from the DataFrame
df = df.drop(["day", "timestamp"], axis=1)

# Displaying the first 3 rows of the modified DataFrame
df.head(3)

## <span style="color:#ff5f27;"> 🤖 Making the predictions </span>

In [ ]:
# Using the serving function to make predictions for the given DataFrame
x = serving_function(
    tf.constant(
        df.values.reshape(-1, df.shape[0], df.shape[1]), tf.float32)
)

# Extracting and converting the predictions to a NumPy array
prediction = x[list(x.keys())[0]].numpy() #.flatten().tolist()
prediction

In [ ]:
# Extracting and organizing the predicted prices for different regions from the batch predictions
price_se1_pred = []
price_se2_pred = []
price_se3_pred = []
price_se4_pred = []

for batch_n in range(len(prediction)):
    batch = prediction[batch_n]
    for window_n in range(df.shape[0]):
        price_se1_pred.append(batch[window_n][0])
        price_se2_pred.append(batch[window_n][1])
        price_se3_pred.append(batch[window_n][2])
        price_se4_pred.append(batch[window_n][3])

In [ ]:
# Initializing serving
feature_view.init_serving(1)

# Accessing the transformation functions used in the serving configuration
fv_transformation_functions = feature_view._vector_server.model_dependent_transformation_functions

In [ ]:
# Decoding the features from the predicted prices
preds = pd.DataFrame(data={'price_se1': price_se1_pred,
                           'price_se2': price_se2_pred,
                           'price_se3': price_se3_pred,
                           'price_se4': price_se4_pred,})

# Extracting and decoding the transformation functions used in serving
res = {}
for transformation_function in fv_transformation_functions:
    udf = transformation_function.hopsworks_udf
    transformed_features = udf.transformation_features[0]
    if transformed_features in ["price_se1", "price_se2", "price_se3", "price_se4"] and udf.function_name == "min_max_scaler":
        stats = udf.transformation_statistics
        preds[transformed_features] = preds[transformed_features].map(lambda x: x*(stats.feature.max-stats.feature.min)+stats.feature.min)

# Applying a transformation to reverse the sign of the decoded features
preds = preds.apply(lambda x: -x)
preds

In [ ]:
# Plotting the predicted electricity prices for different regions
plt.plot(preds["price_se1"],color='red', label='SE1 price prediction')
plt.plot(preds["price_se2"], color='blue', label='SE2 price prediction')
plt.plot(preds["price_se3"], color='green', label='SE3 price prediction')
plt.plot(preds["price_se4"], color='black', label='SE4 price prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend(loc='upper left')

# Adjusting the size of the plot
fig = plt.gcf()
fig.set_size_inches(15, 5)

---

### <span style="color:#ff5f27;">🥳 <b> Next Steps  </b> </span>
Congratulations you've now completed the Electricity price tutorial for Managed Hopsworks.

Check out our other tutorials on ➡ https://github.com/logicalclocks/hopsworks-tutorials

Or documentation at ➡ https://docs.hopsworks.ai